In [587]:
import pandas as pd

random_state = 54278

df = pd.read_csv("dataset6.csv")

In [588]:
cols = df.columns
cols = cols.drop("churn")
print(cols)

Index(['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',
       'callcard', 'wireless', 'longmon', 'tollmon', 'equipmon', 'cardmon',
       'wiremon', 'longten', 'tollten', 'cardten', 'voice', 'pager',
       'internet', 'callwait', 'confer', 'ebill', 'loglong', 'logtoll',
       'lninc', 'custcat'],
      dtype='object')


In [589]:
df.head()

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,...,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,4.40,...,1.0,0.0,1.0,1.0,0.0,1.482,3.033,4.913,4.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,9.45,...,0.0,0.0,0.0,0.0,0.0,2.246,3.240,3.497,1.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,6.30,...,0.0,0.0,0.0,1.0,0.0,1.841,3.240,3.401,3.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,6.05,...,1.0,1.0,1.0,1.0,1.0,1.800,3.807,4.331,4.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,7.10,...,0.0,0.0,1.0,1.0,0.0,1.960,3.091,4.382,3.0,0.0


In [590]:
corr = df.corr()
print(corr["churn"])

tenure     -0.376860
age        -0.287697
address    -0.260659
income     -0.090790
ed          0.216112
employ     -0.337969
equip       0.275284
callcard   -0.311451
wireless    0.174356
longmon    -0.292026
tollmon     0.014356
equipmon    0.255348
cardmon    -0.177163
wiremon     0.177483
longten    -0.294906
tollten    -0.059482
cardten    -0.228634
voice       0.142315
pager       0.124623
internet    0.254838
callwait   -0.052885
confer     -0.081361
ebill       0.254838
loglong    -0.336869
logtoll    -0.070156
lninc      -0.117459
custcat     0.014839
churn       1.000000
Name: churn, dtype: float64


In [591]:
X = df.drop("churn", axis=1)
y = df["churn"]

In [592]:
def scale_standard(X):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler() 
    X = scaler.fit_transform(X)
    return X

def scale_min_max(X):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler() 
    X = scaler.fit_transform(X)
    return X

# X = scale_standard(X)
X = scale_min_max(X)

In [593]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=random_state)

X_train = pd.DataFrame(data=X_train, columns=cols)
X_test = pd.DataFrame(data=X_test, columns=cols)
y_train = pd.DataFrame(data=y_train, columns=["churn"])
y_test = pd.DataFrame(data=y_test, columns=["churn"])

In [594]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)
selected_feat= X_train.columns[(sel.get_support())]
print(len(selected_feat))
print(selected_feat)

13
Index(['tenure', 'age', 'address', 'income', 'ed', 'employ', 'longmon',
       'equipmon', 'cardmon', 'longten', 'cardten', 'loglong', 'lninc'],
      dtype='object')


In [595]:
X_train = X_train[selected_feat]
X_test = X_test[selected_feat]

In [596]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(random_state=random_state, C=0.5, solver="newton-cg")
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

In [597]:
from sklearn.metrics import accuracy_score, log_loss

print(f"{accuracy_score(y_test, y_pred)*100}% acc", end="")
print(f" / {log_loss(y_test, y_pred)} log loss")

72.5% acc / 9.498183498536193 log loss


# Results with all features (C default, solver default)
## Non scaled data: 75% acc / 8.645 log loss
## Standardized data: 80% acc / 6.907 log loss 
## Normalized data: 75.0% acc / 8.634 log loss

# Results with features from Forest (C default, solver default)
## Non scaled data: 67.5% acc / 11.225 log loss
## Standardized data: 70% acc / 10.361 log loss
## Normalized data: 72.5% acc / 9.498 log loss

# Results with features from Forest (C 0.5, solver liblinear)
## Non scaled data: 75% acc / 8.634 log loss
## Standardized data: 72.5% acc / 9.498 log loss
## Normalized data: 70% acc / 10.361 log loss

# Results with features from Forest (C 0.5, solver newton-cg)
## Non scaled data: 72.5% acc / 9.498 log loss
## Standardized data: 70.0% acc / 10.361 log loss
## Normalized data: 72.5% acc / 9.498 log loss